In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [18]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os


# Define the paths to the dataset
train_path = "../data/CINIC10/train"
test_path = "../data/CINIC10/test"
val_path = "../data/CINIC10/valid"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),  # Resize images to 32x32
        transforms.ToTensor(),  # Convert images to PyTorch tensors
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalize the images
    ]
)

In [20]:
class CustomImageFolder(ImageFolder):
    def __getitem__(self, index):
        # Override the default __getitem__ method to return the image and the folder name
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        # Get the folder name as the label
        class_name = os.path.basename(os.path.dirname(path))
        return sample, class_name


# Usage example
# train_dataset = CustomImageFolder(root=train_path, transform=transform)
# test_dataset = CustomImageFolder(root=test_path, transform=transform)
# val_dataset = CustomImageFolder(root=val_path, transform=transform)
    


In [21]:
from src.CNN import *
from src.augmentations import *

In [22]:
grid = {
    'kernel_sizes': [
        [3], 
        [3, 3],
        [3, 3, 3],
        ],
    'num_filters': [32],
    'num_fc_layers': [2, 3],
    'fc_size': [8, 16],
}

In [23]:
# make each combination of hyperparameters
from itertools import product
keys, values = zip(*grid.items())
combinations = [dict(zip(keys, v)) for v in product(*values)]
len(combinations)

# reverse the combinations
combinations = combinations[::-1]


In [24]:
from tqdm.notebook import tqdm
train_dataset = ImageFolder(root=train_path, transform=transform)
test_dataset = ImageFolder(root=test_path, transform=transform)
val_dataset = ImageFolder(root=val_path, transform=transform)

# Define dataloaders
batch_size = 64*4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

import pandas as pd
try:
    results = pd.read_csv('results.csv')
except:
    results = pd.DataFrame(columns=['kernel_sizes', 'num_filters', 'num_fc_layers', 'fc_size', 'accuracy', 'iteration', 
                                'class0_accuracy', 'class1_accuracy', 'class2_accuracy', 'class3_accuracy', 'class4_accuracy', 
                                'class5_accuracy', 'class6_accuracy', 'class7_accuracy', 'class8_accuracy',
                                'class9_accuracy', ])
    results.to_csv('results.csv', index=False)

EPOCHS = 20
iterations = 5

# get already tested combinations
tested_combinations = results[['kernel_sizes', 'num_filters', 'num_fc_layers', 'fc_size', 'iteration']]

# check if each combination has been tested iterations times
tested_combinations = tested_combinations.groupby(['kernel_sizes', 'num_filters', 'num_fc_layers', 'fc_size']).filter(lambda x: len(x) == iterations)

# remove tested combinations from the list
combinations = [combination for combination in combinations if combination not in tested_combinations.to_dict(orient='records')]

print(f'Number of combinations to train and test: {len(combinations)}')

ten_percent = 90000/batch_size // 10

for combination in tqdm(combinations, desc=f'Combinations'):
    print(f'Combination: {combination}')
    for iteration in tqdm(range(iterations), desc=f'Iteration'):
        net = create_cnn(**combination)
        net = net.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(net.parameters(), lr=0.001, )

        # Train the network
        # losses = []
        for epoch in range(EPOCHS):  # loop over the dataset multiple times
            running_loss = 0.0
            for i, data in enumerate(train_loader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                optimizer.zero_grad()

                outputs = net(inputs)
                # print(outputs.shape)
                # print(labels)
                # break

                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                if i % ten_percent == ten_percent - 1:  
                    print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / ten_percent))
                    # losses.append(running_loss / ten_percent)
                    running_loss = 0.0
                
                    # # if 3 previous losses are not an improvement, break
                    # if len(losses) > 3 and losses[-1] > losses[-2] > losses[-3]:
                    #     print('Early stopping due to loss not improving for 3 batches.')
                    #     break


        print('Finished Training')
        print(f'LOSS {running_loss} for combination {combination} and iteration {iteration}')

        # Test the network
        correct = 0
        total = 0
        class_correct = list(0. for i in range(10))
        class_total = list(0. for i in range(10))
        with torch.no_grad():
            for data in test_loader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                for i in range(10):
                    label = labels[i]
                    class_correct[label] += (predicted[i] == label).item()
                    class_total[label] += 1
            
        results = pd.concat([results, pd.DataFrame([{
            'kernel_sizes': combination['kernel_sizes'], 
            'num_filters': combination['num_filters'], 
            'num_fc_layers': combination['num_fc_layers'], 
            'fc_size': combination['fc_size'], 
            'accuracy': 100 * correct / total, 
            'iteration': iteration,
            'class0_accuracy': 100 * class_correct[0] / class_total[0], 
            'class1_accuracy': 100 * class_correct[1] / class_total[1], 
            'class2_accuracy': 100 * class_correct[2] / class_total[2], 
            'class3_accuracy': 100 * class_correct[3] / class_total[3], 
            'class4_accuracy': 100 * class_correct[4] / class_total[4], 
            'class5_accuracy': 100 * class_correct[5] / class_total[5], 
            'class6_accuracy': 100 * class_correct[6] / class_total[6], 
            'class7_accuracy': 100 * class_correct[7] / class_total[7], 
            'class8_accuracy': 100 * class_correct[8] / class_total[8], 
            'class9_accuracy': 100 * class_correct[9] / class_total[9], 
        }])])

        results.to_csv('results.csv', index=False)

                

        print('Accuracy of the network on the test images: %d %%' % (
            100 * correct / total))

        # Save the model
        combination_str = '_'.join([f'{k}_{v}' for k, v in combination.items()])
        torch.save(net.state_dict(), f'../models/cnn_{combination_str}.pth')



        print(f'Finished Testing for combination {combination} and iteration {iteration}')

Number of combinations to train and test: 12


Combinations:   0%|          | 0/12 [00:00<?, ?it/s]

Combination: {'kernel_sizes': [3, 3, 3], 'num_filters': 32, 'num_fc_layers': 3, 'fc_size': 16}


Iteration:   0%|          | 0/5 [00:00<?, ?it/s]